In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
!pip install scikit-learn



In [2]:
DATASET_DIR = r"C:\Users\kumar\Downloads\potato_desease\training\PlantVillage"

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16


In [3]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)


In [4]:
train_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)


Found 1722 images belonging to 3 classes.
Found 430 images belonging to 3 classes.


In [5]:
import os
print(os.path.exists(DATASET_DIR))
print(os.listdir(DATASET_DIR))


True
['Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight']


In [6]:
print(train_generator.class_indices)


{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}


In [7]:
from sklearn.utils.class_weight import compute_class_weight

classes = train_generator.classes

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(classes),
    y=classes
)

class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)


Class weights: {0: 0.7175, 1: 0.7175, 2: 4.704918032786885}


In [8]:
import sys
!{sys.executable} -m pip install scikit-learn


In [9]:
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False  # FIRST PHASE


In [10]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(3, activation="softmax")
])


In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    class_weight=class_weights
)


Epoch 1/10


108/108 [==============================] - 83s 717ms/step - loss: 0.3641 - accuracy: 0.8746 - val_loss: 0.0979 - val_accuracy: 0.9605
Epoch 2/10
108/108 [==============================] - 85s 791ms/step - loss: 0.1598 - accuracy: 0.9419 - val_loss: 0.0853 - val_accuracy: 0.9651
Epoch 3/10
108/108 [==============================] - 87s 804ms/step - loss: 0.1616 - accuracy: 0.9321 - val_loss: 0.0987 - val_accuracy: 0.9651
Epoch 4/10
108/108 [==============================] - 75s 691ms/step - loss: 0.1081 - accuracy: 0.9570 - val_loss: 0.0495 - val_accuracy: 0.9837
Epoch 5/10
108/108 [==============================] - 75s 699ms/step - loss: 0.1007 - accuracy: 0.9599 - val_loss: 0.1335 - val_accuracy: 0.9535
Epoch 6/10
108/108 [==============================] - 76s 704ms/step - loss: 0.0919 - accuracy: 0.9681 - val_loss: 0.0802 - val_accuracy: 0.9698
Epoch 7/10
108/108 [==============================] - 97s 900ms/step - loss: 0.0630 - accuracy: 0.9727 - val_loss: 0.0446 - val_

In [12]:
import sys
!{sys.executable} -m pip install scipy


In [11]:
base_model.trainable = True

for layer in base_model.layers[:100]:
    layer.trainable = False


In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [13]:
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    class_weight=class_weights
)


Epoch 1/5


108/108 [==============================] - 150s 1s/step - loss: 1.0697 - accuracy: 0.5732 - val_loss: 0.7153 - val_accuracy: 0.6651
Epoch 2/5
108/108 [==============================] - 124s 1s/step - loss: 0.5778 - accuracy: 0.7451 - val_loss: 0.5223 - val_accuracy: 0.7907
Epoch 3/5
108/108 [==============================] - 127s 1s/step - loss: 0.4452 - accuracy: 0.8031 - val_loss: 0.3858 - val_accuracy: 0.8605
Epoch 4/5
108/108 [==============================] - 1262s 12s/step - loss: 0.3366 - accuracy: 0.8502 - val_loss: 0.3044 - val_accuracy: 0.8953
Epoch 5/5
108/108 [==============================] - 134s 1s/step - loss: 0.2946 - accuracy: 0.8699 - val_loss: 0.2345 - val_accuracy: 0.9116


In [14]:
model.save("models/potato_model/1")


INFO:tensorflow:Assets written to: models/potato_model/1\assets


INFO:tensorflow:Assets written to: models/potato_model/1\assets


In [21]:
import tensorflow as tf

model = tf.keras.models.load_model("models/potato_model/1")
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 8, 8, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1280)              5120      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                        